In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import json
import csv
import os
pd.set_option('display.max_rows', None)

In [ ]:
#crash_data = pd.read_csv(("Traffic_Accidents_20231015.csv"), dtype=object)
#crash_data

In [ ]:
# csvpath = os.path.join("Traffic_Accidents_20231015.csv")
# with open('Traffic_Accidents_20231015.csv') as csvfile:
#     crash_data = csv.reader(csvfile, delimiter=',')
#     csv_header = next(crash_data)
#     print(f"CSV Header: {csv_header}")

#       #Read each row of data after the header
#     for row in crash_data:
#         print(row)

In [52]:
url = "https://data.nashville.gov/resource/6v6w-hpcw.json?$limit=140100"
crash_data = requests.get(url).json()
crash_data

[{'accident_number': '20230610811',
  'date_and_time': '2023-10-13T21:10:00.000',
  'number_of_motor_vehicles': '1',
  'number_of_injuries': '0',
  'number_of_fatalities': '0',
  'property_damage': True,
  'hit_and_run': True,
  'reporting_officer': '352170',
  'collision_type': '0',
  'collision_type_description': 'NOT COLLISION W/MOTOR VEHICLE-TRANSPORT',
  'weather': '21',
  'weather_description': 'CLEAR',
  'illuaccidemination': '3',
  'illumination_description': 'DARK - LIGHTED',
  'harmfulcodes': '18',
  'harmfuldescriptions': 'OTHER OBJECT (NOT FIXED)',
  'street_address': 'HAYWOOD LN & HAYWOOD SQUARE PVTDR',
  'city': 'NASHVILLE',
  'state': 'TN',
  'zip': '37211',
  'rpa': '8709',
  'precinct': 'SOUTH',
  'lat': '36.0694',
  'long': '-86.6927',
  'mapped_location': {'type': 'Point', 'coordinates': [-86.6927, 36.0694]},
  ':@computed_region_wvby_4s8j': '1',
  ':@computed_region_3aw5_2wv7': '1',
  ':@computed_region_p6sk_2acq': '28',
  ':@computed_region_gxvr_9jxz': '15'},
 {'ac

In [53]:
crash_data_df = pd.DataFrame(crash_data)
crash_data_df.tail()

,accident_number,date_and_time,number_of_motor_vehicles,number_of_injuries,number_of_fatalities,property_damage,hit_and_run,reporting_officer,collision_type,collision_type_description,...,zip,rpa,precinct,lat,long,mapped_location,:@computed_region_wvby_4s8j,:@computed_region_3aw5_2wv7,:@computed_region_p6sk_2acq,:@computed_region_gxvr_9jxz
140095,20180000173,2018-01-01T01:00:00.000,1.0000,0.0000,0.0000,True,True,493301,0,NOT COLLISION W/MOTOR VEHICLE-TRANSPORT,...,37221,4805,WEST,35.9994,-87.0192,"{'type': 'Point', 'coordinates': [-87.0192, 35...",1,2,35,20
140096,20180000167,2018-01-01T01:12:00.000,2.0000,1.0000,0.0000,NaN,True,212342,9,UNKNOWN,...,37211,8445,SOUTH,36.0809,-86.7267,"{'type': 'Point', 'coordinates': [-86.7267, 36...",1,1,12,3
140097,20180000109,2018-01-01T00:39:00.000,1.0000,0.0000,0.0000,True,True,169413,0,NOT COLLISION W/MOTOR VEHICLE-TRANSPORT,...,37189,3229,NORTH,36.2657,-86.8309,"{'type': 'Point', 'coordinates': [-86.8309, 36...",1,2,1,6
140098,20180000088,2018-01-01T00:25:00.000,3.0000,0.0000,0.0000,NaN,True,226175,5,SIDESWIPE - SAME DIRECTION,...,37209,5023,WEST,36.1447,-86.8777,"{'type': 'Point', 'coordinates': [-86.8777, 36...",1,1,24,46
140099,20180000079,2018-01-01T00:26:00.000,2.0000,0.0000,0.0000,NaN,False,279117,5,SIDESWIPE - SAME DIRECTION,...,37203,4085,CENTRA,36.1592,-86.7814,"{'type': 'Point', 'coordinates': [-86.7814, 36...",1,1,20,40


In [57]:
clean_crash_df = pd.DataFrame(crash_data_df, columns=["accident_number", "date_and_time", "number_of_motor_vehicles",
                                                      "number_of_injuries", "number_of_fatalities", "weather_description", "zip", "precinct"])
clean_crash_df = clean_crash_df.loc[clean_crash_df["accident_number"] < "20230000000"]
clean_crash_df.head()

,accident_number,date_and_time,number_of_motor_vehicles,number_of_injuries,number_of_fatalities,weather_description,zip,precinct
13523,20220663726,2022-12-31T23:50:00.000,2,0,0,FOG,37013,SOUTH
13524,20220663675,2022-12-21T21:39:00.000,2,0,0,CLEAR,37204,MIDTOW
13525,20220663674,2022-12-31T23:20:00.000,1,0,0,CLEAR,37207,NORTH
13526,20220663673,2022-12-31T15:20:00.000,2,0,0,CLEAR,37013,SOUTH
13527,20220663596,2022-12-31T22:35:00.000,2,1,0,CLEAR,37211,SOUTH


In [58]:
total_crashes = len(clean_crash_df)
total_crashes

126577